In [52]:
import pickle, gzip, numpy

#dataset format: touple cu 2 elemente:
# 1. un array cu cate un array de 784 de elemente (pt imagine)
# 2. un array cu labels

#pentru a puteaa itera prin touple vom folosi functia zip 
# zip() returns a zip object, which is an iterator of tuples
#astfel se cupleaza prima imagine cu primul label etc.

#Online Training
def learn(train_set):
    TrainingSet = zip(train_set[0], train_set[1])
    
    b = numpy.random.randn() #un nr random
    w = numpy.random.randn(784)  #784 de valori random adica pt fiecare element din vectorul imaginii
    
    nrIterations = 1
    allClassified = False

    while not allClassified and nrIterations > 0:
        allClassified = True
        for x, t in TrainingSet:         #folosim numpy.dot si numpy.add - Dot product/addition of two arrays-pt produs scalar
            z = numpy.add(numpy.dot(w, x), b)  # produsul scalar + biasul - z = w * x + b
            if z>0:
                output=1
            else:
                output=0  #classify the sample
            w = w + (t - output) * x * 0.1  #adjust the weights 
            b = b + (t - output) * 0.1  #adjust the bias  #learning rate - ul = 0.1

            if output != t:
                allClassified = False

        nrIterations -= 1

    return w, b  #returnam weights si biasul pt fiecare perceptron pt a le folosi la produs scalar + bias cand facem testarea 
    

def main():
    #load dataset
    with gzip.open('mnist.pkl.gz', 'rb') as fin:
        train_set, valid_set, test_set = pickle.load(fin, encoding='latin1')
        
     #lista cu 10 elem https://stackoverflow.com/questions/10712002/create-an-empty-list-in-python-with-certain-size    
    perceptrons = list(range(10))
    
    #pentru fiecare cifra in parte vom avea un perceptron pe care il vom antrena cu setul de date primit astfel:
    #pentru fiecare cifra, daca labelul unui element din dataset este egal cu cifra atunci in datasetul personalizat pentru
    #fiecare perceptron va fi adaugat imaginea iar labelul imaginii va fi 1 (adica acea imagine reprezinta cifra respectiva)
    #daca labelul elementului nu este egal cu cifra pentru care antrenam perceptronul, vom adauga imaginea in datasetul 
    #personalizat insa cu labelul 0 (adica imaginea nu reprezinta cifra selectiei curente)
    for cifra in range (0,10):
        train=() #train setul pentru fiecare cifra in parte
        image_set=[]  #setul de imagini
        label_set=[]  #setul de labeluri pentru imagini
        poz = -1 #pt identificarea pozitiei curente in multimea de elemente din vectorul cu imaginii si din vectorul cu labels
        for label in train_set[1]: #train_set[1] - vectorul cu labels                           # (ex. imaginea 3 cu labelul 3)
            poz=poz+1
            if label==cifra:
                image_set.append(train_set[0][poz])
                label_set.append(1) #punem label 1 pentru ca imaginea reprezinta cifra selectiei curente
            else:
                image_set.append(train_set[0][poz])
                label_set.append(0)
        train=(image_set,label_set)
        #aici antrenam perceptronii
        perceptrons[cifra]=learn(train)
        
  
    all=len(test_set[1]) #numarul total de imagini
    corect=0
           
 
    test = zip(test_set[0], test_set[1]) #pt a putea itera prin setul de test

# fiecare image vector il vom trece prin fiecare perceptron
#perceptronul ales va fi cel cu z mai mare
    for image, labelImage in test:
        maximZ = float('-inf')      #https://stackoverflow.com/questions/7604966/maximum-and-minimum-values-for-ints
        perceptronLabel = -1     
        for cifra, perceptron in enumerate(perceptrons):  #cifra perceptronului si perceptronul curent
            weight = perceptron[0]
            b = perceptron[1]
            z = numpy.add(numpy.dot(weight, image), b) #care produs scalar e mai mare, pe acel perceptron il alegem
            if z > maximZ:
                maximZ = z
                perceptronLabel = cifra  #retinem care perceptron e ales
        if perceptronLabel == labelImage:  #daca labelul imaginii este egal cu cifra perceptronului avem o prezicere corecta
            corect=corect+1  #numaram cate imagini au fost corect clasificate

    print("Accuracy:")  #acuratetea o calculam numarul de imagini corect clasificate impartit la numarul total de imagini
    print((corect/all)*100)  #pentru a obtine un rezultat intre 0-100 inmultim rezultatul cu 100
    
        
    
if __name__ == '__main__':
    main()

Accuracy:
87.83
